In [1]:
import nltk
import pandas as pd
import spacy
from sklearn.utils import shuffle

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


True

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

C:\Users\HP\Anaconda3\envs\nlp_course\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
sia = SentimentIntensityAnalyzer()

In [4]:
sia.polarity_scores('i am really sad')

{'neg': 0.629, 'neu': 0.371, 'pos': 0.0, 'compound': -0.5256}

In [5]:
# data = pd.read_csv('./clean_tweet.csv.csv',  encoding='ISO-8859-1', names=['sentiment', 'ids', 'date','flag', 'user','text'])
# #(0 = negative, 2 = neutral, 4 = positive)
# data = data.dropna()
# data = data.drop(columns=['ids','date','flag','user'])
# # print(len(data))
# data.sentiment = data.sentiment.replace(4,1)
# data = shuffle(data, random_state=22)
# data.head()
# # data.to_csv('./dataFinal.csv', index= False)
# # data.sentiment.value_counts()

In [6]:
data1 = pd.read_csv('./clean_tweet.csv', index_col=0)
data1 = data1.dropna()
data1.target = data1.target.replace(4,1)
data1 = shuffle(data1, random_state=22)
data1.head()

C:\Users\HP\Anaconda3\envs\nlp_course\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,text,target
1200205,i m honored my friend have a great week,1
1097971,hahas gotta change airline,1
306692,grrrrrrrr my picture isn t showing up,0
94044,el pollo loco commercials gross me out i can j...,0
1211014,see wouldnt it have been easier if you were on...,1


In [8]:
import re
def preprocess(tweet):
    #Preprocess the text in a single tweet
    #arguments: tweet = a single tweet in form of string 
    #convert the tweet to lower case
    tweet.lower()
    #convert all urls to sting "URL"
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #convert all @username to "AT_USER"
    tweet = re.sub('@[^\s]+','AT_USER', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
    #convert "#topic" to just "topic"
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    return tweet



data1['text'] = data1['text'].apply(preprocess)



In [9]:
data1['sss'] = data1['text'].apply(lambda text: sia.polarity_scores(text))

In [10]:
data1['scompund'] = data1['sss'].apply(lambda s: s['compound'])

In [11]:
data1['pedicted'] = data1['scompund'].apply(lambda sss : 1 if sss >= 0 else 0 )

In [29]:
data1.head()

,text,target,sss,scompund,pedicted
1200205,i m honored my friend have a great week,1,"{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'comp...",0.9022,1
1097971,hahas gotta change airline,1,"{'neg': 0.0, 'neu': 0.517, 'pos': 0.483, 'comp...",0.4215,1
306692,grrrrrrrr my picture isn t showing up,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,1
94044,el pollo loco commercials gross me out i can j...,0,"{'neg': 0.205, 'neu': 0.795, 'pos': 0.0, 'comp...",-0.4767,0
1211014,see wouldnt it have been easier if you were on...,1,"{'neg': 0.0, 'neu': 0.766, 'pos': 0.234, 'comp...",0.7003,1


In [16]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [17]:
accuracy_score(data1['target'],data1['pedicted'])

0.6609619432508719

In [19]:
print(classification_report(data1['target'],data1['pedicted']))

              precision    recall  f1-score   support

           0       0.80      0.42      0.56    798502
           1       0.61      0.90      0.73    798245

   micro avg       0.66      0.66      0.66   1596747
   macro avg       0.71      0.66      0.64   1596747
weighted avg       0.71      0.66      0.64   1596747



In [22]:
print(confusion_matrix(data1['target'],data1['pedicted']))

[[339362 459140]
 [ 82218 716027]]
